In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam

import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
import time

import os
import fnmatch

In [2]:
x_train = np.zeros((40479+17000,139,139,3), np.float32)
#x_test = []
y_train = []

df_train = pd.read_csv('train_v2.csv')
#df_test = pd.read_csv('../input/sample_submission.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

#files = []

#for f, tags in tqdm(df_train.values[:40479], miniters=1000):
#    files.append([i for i in os.listdir('train-augmented') if os.path.isfile(os.path.join('train-augmented',i)) and \
#         f in i])
    
#for file in os.listdir('train-augmented'):
#    if fnmatch.fnmatch(file, '*.txt'):
#        print file
    

files = os.listdir('train-augmented')
file_dict = {}
for file in files:
    s = file.split('_')
    if s[0]+'_'+s[1] in file_dict:
        file_dict[s[0]+'_'+s[1]].append(file)
    else:
        file_dict[s[0]+'_'+s[1]] = [file]

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    #x_train.append(img)    
    x_train[i,:,:,:] = np.array(cv2.resize(img, (139, 139)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
    if f in file_dict:
        for file in file_dict[f]:
            img = cv2.imread('train-augmented/'+file)
            x_train[i,:,:,:] = np.array(cv2.resize(img, (139, 139)),np.float32)/255.#139 minimum size for inception
            i+=1           
            y_train.append(targets)
#    for g in files[i]:
#        img = cv2.imread(('train-augmented/'+g)) 
#        x_train.append(cv2.resize(img, (139, 139)))#139 minimum size for inception
#        y_train.append(targets)
#    i+=1

#for f, tags in tqdm(df_test.values, miniters=1000):
#    img = cv2.imread('../input/test-jpg/{}.jpg'.format(f))
#    x_test.append(cv2.resize(img, (32, 32)))
    
y_train = np.array(y_train, np.uint8)
#x_train = np.asarray(x_train, np.float32) / 255.
#x_test  = np.array(x_test, np.float32) / 255.

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:38<00:00, 410.26it/s]

(57479, 139, 139, 3)
(57479, 17)


In [3]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train = x_train - train_mean
x_val = x_val - train_mean

(51731, 139, 139, 3)
(51731, 17)
(5748, 139, 139, 3)
(5748, 17)


In [5]:
base_model = InceptionV3(input_shape=(139,139,3), weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#fully connected layer
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(17, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    

In [6]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=20,callbacks=callbacks,
                  shuffle=True)

Train on 51731 samples, validate on 5748 samples
Epoch 1/20
181s - loss: 0.2108 - acc: 0.9210 - val_loss: 0.1765 - val_acc: 0.9308
Epoch 2/20
177s - loss: 0.1759 - acc: 0.9309 - val_loss: 0.1667 - val_acc: 0.9348
Epoch 3/20
177s - loss: 0.1694 - acc: 0.9332 - val_loss: 0.1604 - val_acc: 0.9369
Epoch 4/20
177s - loss: 0.1648 - acc: 0.9350 - val_loss: 0.1565 - val_acc: 0.9380
Epoch 5/20
177s - loss: 0.1618 - acc: 0.9360 - val_loss: 0.1566 - val_acc: 0.9380
Epoch 6/20
177s - loss: 0.1576 - acc: 0.9377 - val_loss: 0.1535 - val_acc: 0.9396
Epoch 7/20
177s - loss: 0.1547 - acc: 0.9385 - val_loss: 0.1537 - val_acc: 0.9393
Epoch 8/20
177s - loss: 0.1523 - acc: 0.9394 - val_loss: 0.1518 - val_acc: 0.9399
Epoch 9/20
177s - loss: 0.1501 - acc: 0.9402 - val_loss: 0.1494 - val_acc: 0.9414
Epoch 10/20
177s - loss: 0.1472 - acc: 0.9413 - val_loss: 0.1470 - val_acc: 0.9426
Epoch 11/20
177s - loss: 0.1441 - acc: 0.9425 - val_loss: 0.1469 - val_acc: 0.9429
Epoch 12/20
177s - loss: 0.1429 - acc: 0.9428 -

In [9]:
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=10,callbacks=callbacks,
                  shuffle=True)

Train on 51731 samples, validate on 5748 samples
Epoch 1/10
178s - loss: 0.1192 - acc: 0.9521 - val_loss: 0.1397 - val_acc: 0.9470
Epoch 2/10
176s - loss: 0.1168 - acc: 0.9530 - val_loss: 0.1392 - val_acc: 0.9472
Epoch 3/10
177s - loss: 0.1149 - acc: 0.9536 - val_loss: 0.1389 - val_acc: 0.9477
Epoch 4/10
177s - loss: 0.1129 - acc: 0.9544 - val_loss: 0.1382 - val_acc: 0.9481
Epoch 5/10
176s - loss: 0.1114 - acc: 0.9551 - val_loss: 0.1384 - val_acc: 0.9477
Epoch 6/10
176s - loss: 0.1103 - acc: 0.9553 - val_loss: 0.1398 - val_acc: 0.9479
Epoch 7/10
176s - loss: 0.1086 - acc: 0.9562 - val_loss: 0.1398 - val_acc: 0.9483


In [10]:
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=10,callbacks=callbacks,
                  shuffle=True)

Train on 51731 samples, validate on 5748 samples
Epoch 1/10
178s - loss: 0.1028 - acc: 0.9586 - val_loss: 0.1378 - val_acc: 0.9484
Epoch 2/10
176s - loss: 0.1016 - acc: 0.9590 - val_loss: 0.1384 - val_acc: 0.9488
Epoch 3/10
177s - loss: 0.1008 - acc: 0.9593 - val_loss: 0.1383 - val_acc: 0.9489
Epoch 4/10
177s - loss: 0.1000 - acc: 0.9596 - val_loss: 0.1388 - val_acc: 0.9489


In [11]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

#we need to recompile the model for these modifications to take effect
#Adam with low learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=10,callbacks=callbacks,
                  shuffle=True)

Train on 51731 samples, validate on 5748 samples
Epoch 1/10
272s - loss: 0.1113 - acc: 0.9559 - val_loss: 0.1226 - val_acc: 0.9530
Epoch 2/10
267s - loss: 0.0901 - acc: 0.9640 - val_loss: 0.1202 - val_acc: 0.9561
Epoch 3/10
267s - loss: 0.0755 - acc: 0.9698 - val_loss: 0.1205 - val_acc: 0.9574
Epoch 4/10
268s - loss: 0.0647 - acc: 0.9740 - val_loss: 0.1233 - val_acc: 0.9577
Epoch 5/10
268s - loss: 0.0554 - acc: 0.9780 - val_loss: 0.1250 - val_acc: 0.9595


In [12]:
#lower learning rate further
model.compile(optimizer=Adam(lr=0.00005), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=10,callbacks=callbacks,
                  shuffle=True)

Train on 51731 samples, validate on 5748 samples
Epoch 1/10
272s - loss: 0.0429 - acc: 0.9831 - val_loss: 0.1282 - val_acc: 0.9600
Epoch 2/10
268s - loss: 0.0340 - acc: 0.9867 - val_loss: 0.1368 - val_acc: 0.9601
Epoch 3/10
268s - loss: 0.0301 - acc: 0.9883 - val_loss: 0.1375 - val_acc: 0.9606
Epoch 4/10
268s - loss: 0.0266 - acc: 0.9898 - val_loss: 0.1494 - val_acc: 0.9605


In [14]:
for thresh in [0.1,0.2,0.3,0.4,0.5]:
    y_pred = model.predict(x_val, batch_size = 128, verbose=2)
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))
    print(np.sum(y_val==(np.array(y_pred)>thresh),axis=0)/y_val.shape[0])#setting different thresholds for different classes      

thresh: 0.1 	F2 score: 0.912377966791
[ 0.88239388  0.99182324  0.98051496  0.88065414  0.99512874  0.98242867
  0.9966945   0.9848643   0.89231037  0.96189979  0.95755045  0.99895616
  0.96572721  0.93302018  0.95250522  0.93006263  0.9032707 ]
thresh: 0.2 	F2 score: 0.906609419596
[ 0.89248434  0.99199722  0.98295059  0.89665971  0.99530271  0.98521225
  0.99721642  0.98573417  0.9032707   0.96677105  0.95981211  0.99930411
  0.97007655  0.93789144  0.96033403  0.93928323  0.91092554]
thresh: 0.3 	F2 score: 0.900366564998
[ 0.894746    0.99199722  0.98382046  0.90588031  0.99634656  0.9848643
  0.9966945   0.98608212  0.90762004  0.96938065  0.95929019  0.99930411
  0.9704245   0.94067502  0.96137787  0.94328462  0.91022965]
thresh: 0.4 	F2 score: 0.89415036213
[ 0.89596381  0.99251914  0.98312457  0.91127349  0.99599861  0.98538622
  0.9966945   0.98695198  0.91283925  0.97059847  0.95946416  0.99913013
  0.97112039  0.9401531   0.96189979  0.94241475  0.91231733]
thresh: 0.5 	F2 sc

/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [15]:
def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

In [18]:
total,tp,tn,fp,fn = multilabelmetrics(y_val,np.array(y_pred)>0.3)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
agriculture,241,364,3472,1671,1912
slash_burn,35,11,5611,91,126
cloudy,35,58,5386,269,304
cultivation,282,259,4732,475,757
artisinal_mine,15,6,5570,157,172
blooming,60,27,5579,82,142
blow_down,9,10,5634,95,104
selective_logging,46,34,5571,97,143
water,232,299,4258,959,1191
primary,46,130,319,5253,5299


In [19]:
model.save("Inception_augmented")

In [20]:
x_train = []
x_val = []
x_test = []
y_train = []
#y_pred = []

#df_train = pd.read_csv('train_v2.csv')
df_test = pd.read_csv('sample_submission_v2.csv')

for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
#    x_test.append(img)
    x_test.append(cv2.resize(img, (139, 139)))
    #x_test =  np.array(cv2.resize(img, (139, 139)),np.float32)/255. - train_mean
    #y_pred.append(model.predict(np.expand_dims(x_test, axis=0)))
#x_test  = np.array(x_test, np.float32) / 255.
#y_pred = np.array(y_pred)
#print(y_pred.shape)
#print(x_test.shape)

100%|██████████| 61191/61191 [01:42<00:00, 595.87it/s]


In [21]:
#prediction for first half of test set (to resolve out of memory issues)
x_test_np  = np.array(x_test[:int(len(x_test)/2)], np.float32) / 255.
x_test_np = x_test_np - train_mean
y_pred_1 = model.predict(x_test_np, batch_size = 128, verbose=2)
#prediction for second half of test set (to resolve out of memory issues)
x_test_np = []
x_test_np  = np.array(x_test[int(len(x_test)/2):], np.float32) / 255.
x_test_np = x_test_np - train_mean
y_pred_2 = model.predict(x_test_np, batch_size = 128, verbose=2)

y_pred = np.concatenate([y_pred_1,y_pred_2])

print(y_pred.shape)

(61191, 17)


In [26]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:]) > 0.01]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_augmented_0_01.csv', index=False)

In [27]:
subm

,image_name,tags
0,test_0,primary clear
1,test_1,blooming selective_logging primary clear
2,test_2,primary partly_cloudy
3,test_3,primary partly_cloudy clear
4,test_4,cloudy primary partly_cloudy
5,test_5,primary clear
6,test_6,agriculture slash_burn cultivation primary par...
7,test_7,agriculture primary bare_ground clear habitati...
8,test_8,primary clear
9,test_9,agriculture cultivation water primary partly_c...
